In [1]:
import arcgis
from arcgis.gis import GIS
import pandas as pd
import datetime

In [40]:
class CameraTrapSurvey:

    def __init__(self, url: str, username: str, password: str, survey_id: str):
        self.gis = GIS(url=url, username=username, password=password)
        self.survey_manager = arcgis.apps.survey123.SurveyManager(self.gis)
        self.survey_id = survey_id
        self.survey = self.survey_manager.get(self.survey_id)
        self.df = self.survey.download('DF')
        self.df['date'] = self.df['date_and_time_of_camera_setup_o'].dt.date

    def get_survey(self):
        return self.survey

    def get_survey_df(self):
        return self.df

    def get_entry(self, camera_id: str, date: str):

        return self.df.loc[(self.df['date'] == datetime.datetime.strptime(
            date, "%Y-%m-%d").date()) & (self.df['camera_id'] == camera_id)]

    def get_info(self, camera_id: str, date: str):

        survey_entries = self.get_entry(camera_id=camera_id, date=date)
        survey_dict = survey_entries.to_dict()

        x, y = survey_dict['SHAPE'][0].x, survey_dict['SHAPE'][0].y
        project_name = survey_dict['project_name'][0]
        camera_setup_date = survey_dict['date_and_time_of_camera_setup_o'][0]
        procedure = survey_dict['_procedure'][0]
        camera_attached_to = survey_dict['camera_attached_to'][0]
        camera_height = survey_dict['camera_height'][0]
        area_deployed = survey_dict['name_of_area_deployed'][0]
        camera_make = survey_dict['camera_make'][0]
        camera_settings = survey_dict['camera_settings'][0]
        camera_feature = survey_dict['what_feature_is_the_camera_targ'][0]
        camera_trap_test = survey_dict['camera_trap_test'][0]
        camera_working = survey_dict['camera_working_when_you_left_or'][0]

        info_dict = {
            'X': x,
            'Y': y,
            'Camera setup date': camera_setup_date,
            'Project Name': project_name,
            'Procedure': procedure,
            'Camera attached to': camera_attached_to,
            'Camera height': camera_height,
            'Area deployed': area_deployed,
            'Camera make': camera_make,
            'Camera settings': camera_settings,
            'Camera feature': camera_feature,
            'Camera trap test': camera_trap_test,
            'Camera working when left': camera_working
        }

        return info_dict

In [41]:
survey = CameraTrapSurvey(url="https://learngis2.maps.arcgis.com/",
                username="0627130_LearnArcGIS",
                password="arcgispass@123",
                survey_id="bcafa6fa2e584123a4f2715474cf6327")

In [29]:
geo_location = survey.get_entry('Camera1','2022-12-04').SHAPE.to_numpy()[0]
x,y = geo_location['x'], geo_location['y']

In [30]:
x,y

(76.40889146000008, 10.447309711000054)

In [42]:
survey.get_info('Camera1','2022-12-04')

{'X': 76.40889146000008,
 'Y': 10.447309711000054,
 'Camera setup date': Timestamp('2022-12-04 16:43:00'),
 'Project Name': 'One',
 'Procedure': 'New camera placement',
 'Camera attached to': 'Post',
 'Camera height': 3.0,
 'Area deployed': 'Kerala',
 'Camera make': '_Bushnell_',
 'Camera settings': 'Photos',
 'Camera feature': 'Rubbing posts',
 'Camera trap test': 'Yes',
 'Camera working when left': 'Yes'}

In [16]:
survey.get_entry('Camera1','2022-12-04')

,objectid,globalid,CreationDate,Creator,EditDate,Editor,project_name,your_name,camera_id,date_and_time_of_camera_setup_o,...,camera_make,untitled_question_11_other,camera_settings,what_feature_is_the_camera_targ,untitled_question_13_other,camera_trap_test,camera_working_when_you_left_or,comments,SHAPE,date
0,1,230b12bf-6ef2-4d1f-8656-5497bc9b1ba8,2022-08-04 17:13:02.348,0627130_LearnArcGIS,2022-08-04 17:13:02.348,0627130_LearnArcGIS,One,Vaishakh,Camera1,2022-12-04 16:43:00,...,_Bushnell_,None,Photos,Rubbing posts,None,Yes,Yes,Nothing,"{""x"": 76.40889146000008, ""y"": 10.4473097110000...",2022-12-04
